In [1]:
import pandas as pd
import numpy as np
import json
from polyglot.text import Text
from polyglot.downloader import downloader


In [4]:
tweets=pd.read_csv('datasets/facebook/allcomment.csv', index_col=0, encoding='utf8', engine='python')
tweets = tweets[~pd.isnull(tweets.message)]

del tweets['can_remove']
tweets.sample(10)

,id,post_id,page_id,fb_id,message,created_time
1040355,1.726614e+15,10156643871935725,1.530806e+11,1.020863e+16,"Yes, tired of ""media"" debates causing controve...",2016-02-15 08:31:46
1663415,9.243567e+14,924335370956326,8.893079e+14,1.020568e+16,https://www.facebook.com/photo.php?fbid=102057...,2015-06-11 21:05:18
1156720,1.015666e+16,10156655212180725,1.530806e+11,1.535945e+15,Ted Cruz is full of vicious lies and comments....,2016-02-18 11:22:12
4105,9.766945e+06,241443930926,1.530806e+11,1.145239e+15,"DEAR DONALD TRUMP, MY NAME IS KEVIN ZARBAILOV ...",2010-01-16 13:22:12
845099,1.015663e+16,10156626755220725,1.530806e+11,1.020447e+16,YOU ROCK< please read this..........A Man Name...,2016-02-10 23:40:53
1535763,7.351590e+14,735136679874672,1.249556e+14,1.020786e+16,<3,2014-11-05 00:39:24
699063,1.757738e+15,10156613640555725,1.530806e+11,1.020758e+16,I am Italian and not vote for the US elections...,2016-02-07 07:12:28
1650100,9.652456e+14,1021780391210298,1.249556e+14,1.029515e+15,"""Who decided you don't got enough to teach chi...",2016-04-05 03:03:31
1339914,1.015667e+16,10156670359400725,1.530806e+11,1.020337e+16,"Go Trump!!,you're the only one id,want in thete",2016-02-22 06:49:27
108798,3.493647e+07,10152061786825725,1.530806e+11,1.053436e+15,because they can and we as the little people c...,2012-08-30 23:55:20


In [ ]:
print len(tweets)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pylab as plt
%matplotlib inline
text = " ".join([x.encode('utf-8').strip() for x in tweets['message']])
wordcloud = WordCloud(background_color='white').generate(text)
plt.figure(figsize=(15, 20))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
len(text.split(" "))

In [ ]:
def get_sentiment(e):
    try:
        pos=e.positive_sentiment
        neg=e.negative_sentiment
        return pos,neg
    except:
        return '',''

def map_entity(e):
    pos,neg=get_sentiment(e)
    return {'tag':e.tag,'entity':" ".join(e),'pos':pos,'neg':neg}

def get_entites(text):
    try:
        polyglot_text=Text(text)
        entities=polyglot_text.entities
        return map(map_entity,entities)
    except:
        return []
     
def map_to_data(tweet):
    user_name=tweet['user']['name']
    text=tweet['text']
    return {'text':text,'user':user_name,'entities':get_entites(text)}

In [ ]:
tweets['entities']=tweets['message'].map(get_entites)

In [7]:
tweets.groupby('fb_id').count()['id']
# 153080620724  = Donald Trump
# 124955570892789  = Sanders
# 889307941125736  = Clinton
mapp = { 153080620724: 'Donald Trump', 124955570892789: 'Bernie Sanders', 889307941125736: 'Hillary Clinton'}

#tweets['from_name'] = tweets['from_id'].map(mapp)
del tweets['page_name']
tweets.insert(3, 'page_name', tweets['page_id'].map(mapp))
tweets.insert(5, 'fb_name', tweets['fb_id'].map(mapp))
tweets.sample(5)

,id,post_id,page_id,page_name,fb_id,fb_name,message,created_time
1591830,8.481877e+14,847706408617698,1.249556e+14,Bernie Sanders,1.160356e+15,NaN,"""the all"" is not part of their lexicon.",2015-06-09 20:44:13
1497600,6.536537e+14,653482638040077,1.249556e+14,Bernie Sanders,1.015346e+16,NaN,and E. Warren; Willie MUST be kept clear of ...,2014-05-16 07:57:14
1422917,1.015668e+16,10156680114785725,1.530806e+11,Donald Trump,1.020421e+16,NaN,I heard Cruz LIE about Trump on O'Reilly last ...,2016-02-24 11:36:24
241830,3.638109e+07,10152210324590725,1.530806e+11,Donald Trump,1.020078e+16,NaN,You mistyped Mitt Romney's name.,2012-10-28 21:19:43
1223994,1.015666e+16,10156658168535725,1.530806e+11,Donald Trump,1.020718e+16,NaN,"I am Argentinian and Catholic,respect the Pope...",2016-02-19 02:20:43


In [ ]:
# %%timeit 

# print 'asd'
# print 'asd'

In [8]:
print len(tweets[~pd.isnull(tweets.fb_name)])

17

In [9]:
tweets.groupby('fb_name').count()['id']

fb_name
Bernie Sanders     15
Donald Trump        1
Hillary Clinton     1
Name: id, dtype: int64

In [13]:
tweets.groupby(['page_name', 'post_id']).count()['id']

page_name        post_id          
Bernie Sanders   100171176808006         1
                 10151939274160734     268
                 10152125009092893     444
                 10152171937945734      26
                 10152230278035734      36
                 10152237789680734      75
                 10152238374850734      66
                 10152242860875734     385
                 10152244734535734      12
                 10152252604985734      77
                 1021780391210298      965
                 102231426521051        11
                 103174799782891        46
                 103898726477201        34
                 105864562872024        11
                 107982319277730         1
                 108320465995622         8
                 108580025979025        35
                 109264342490607         7
                 109730062441178         7
                 110194529133156         4
                 110260102380720         3
                 11

In [14]:
tweets.groupby(['page_name']).count()['id']

page_name
Bernie Sanders      176035
Donald Trump       1451568
Hillary Clinton     122601
Name: id, dtype: int64

In [20]:
print "Trump: %d" % (1451568/807)
print "Sanders: %d" % (176035/1172) 
print "Hillary: %d" % (122601/201) 

Trump: 1798
Sanders: 150
Hillary: 609


In [16]:
176035/1172 

150

In [17]:
122601/201 

609

In [11]:
tweets.groupby('post_id').count()['id']

post_id
1001350573254805     1283
100171176808006         1
1001769359879593      284
1002360753153787      392
1002737249782804     1160
1005367706186425      469
1006474436075752      438
1006902082699654      457
1006907236032472      672
1009111585812037     1135
1010033315719864      923
1011463998910129      684
1011821582207704      487
1011997835523412      459
1012987625424433      138
1013086488747880      484
101389543260237       145
1014025208654008      850
1014433791946483      570
10150126653070554     125
10150142451675725     147
10150370919630725     414
10150536788685725     526
10150565134900725    1194
10150588071420725    2274
10150757204425725     271
10150816231630725     571
10150868615275725     154
10150872796955725     153
10150876741715725     184
                     ... 
974472185942644       396
976935625696300       234
977254912331038       743
978714835518379       733
981858305204032       594
981911965198666       247
983778715011991       988
9841

In [ ]:
#downloader.download('ner2.sco',download_dir='/root/polyglot_data')

In [ ]:
entities = {}

In [ ]:
# {for k, v tweets['entities']}

In [ ]:
for row_id, tweet in tweets.iterrows():
    for ent in tweet['entities']:
        if ent['entity'] not in entities:
            entities.update({ent['entity']: {}})        

In [ ]:
for row_id, tweet in tweets.iterrows():
    for ent in tweet['entities']:
        if ent['entity'] not in entities:
            entities.update({ent['entity']: {}})        

In [ ]:
#tweets = speeches_2016
candidate_col = 'Actor'
candidates = []
for row_id, tweet in tweets.iterrows():
   cand = tweet[candidate_col]
   for ent in tweet['entities']:
       e = ent['entity']
       if check_if_candidate(e):
           candidates.append((cand, check_if_candidate(e)))
[x for x in candidates if x[0] != x[1]]